#Intalando libs

In [1]:
!pip install Python-CartolaFC

     |████████████████████████████████| 81kB 3.7MB/s 


# Criando base de dados

In [41]:
from cartolafc import Api as ApiCartola
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

NUM_RODADAS = 38
ARQUIVO_PARTIDAS = 'dados_partidas.csv'
ARQUIVO_JOGADORES = 'dados_jogadores.csv'
ARQUIVO_PROCESSADO = 'dados_processados.csv'

EMAIL = None #Digite seu email
PASSWORD = None # Digite sua senha

In [17]:
api_cartola = ApiCartola(email=EMAIL, password=PASSWORD)


In [16]:
def descobre_rodada_atual(df_partidas):
    return int(df_partidas[df_partidas['placar_casa']!=-1].rodada.max()) + 1

def resultado_partida(gols_feitos, gols_sofridos, retorna_nome=False):
    # None
    if (gols_feitos is None) or (gols_sofridos is None):
        if retorna_nome:
            return 'N'
        else:
            return np.nan
    # Nan
    elif (gols_feitos!=gols_feitos) or (gols_sofridos!=gols_sofridos):
        if retorna_nome:
            return 'N'
        else:
            return np.nan
    # Vitoria
    elif gols_feitos>gols_sofridos:
        if retorna_nome:
            return 'V'
        else:
            return 1
    # Derrota
    elif gols_feitos<gols_sofridos:
        if retorna_nome:
            return 'D'
        else:
            return -1
    # Empate
    elif gols_feitos==gols_sofridos:
        if retorna_nome:
            return 'E'
        else:
            return 0
def cria_tabela_de_partidas(api_cartola):
    lista_todas_partidas = []
    for rodada in tqdm(range(1,NUM_RODADAS+1)):
        lista_partidas = api_cartola.partidas(rodada)
        for partida in lista_partidas:
            
            clube_casa = {
                
                'clube_id': partida.clube_casa.id,
                'clube_nome': partida.clube_casa.nome,

                'clube_adversario_id': partida.clube_visitante.id,
                'clube_adversario_nome': partida.clube_visitante.nome,
                
                'gols_sofridos': partida.placar_visitante,
                'gols_feitos': partida.placar_casa,
                
                'resultado_nome': resultado_partida(partida.placar_casa, partida.placar_visitante, retorna_nome=True),
                'resultado_id': resultado_partida(partida.placar_casa, partida.placar_visitante, retorna_nome=False),
                
                'rodada': rodada,
                'local_partida': 'casa',
                'local_partida_id': 0
            }
            
            lista_todas_partidas.append(clube_casa)
            
            clube_visitante = {
                
                'clube_id': partida.clube_visitante.id,
                'clube_nome': partida.clube_visitante.nome,
                
                'clube_adversario_id': partida.clube_casa.id,
                'clube_adversario_nome': partida.clube_casa.nome,
                
                'resultado_nome': resultado_partida(partida.placar_visitante, partida.placar_casa, retorna_nome=True),
                'resultado_id': resultado_partida(partida.placar_visitante, partida.placar_casa, retorna_nome=False),
                
                'gols_sofridos': partida.placar_casa,
                'gols_feitos': partida.placar_visitante,
                
                'rodada': rodada,
                'local_partida': 'visitante',
                'local_partida_id': 1,
            }
            lista_todas_partidas.append(clube_visitante)
            
    df_partidas = pd.DataFrame(lista_todas_partidas)
    df_partidas.to_csv(ARQUIVO_PARTIDAS, index=False)
    return df_partidas

def cria_tabela_jogadores(api_cartola):
    lista_mercado_atletas_formatado = []
    lista_mercado_atletas = api_cartola.mercado_atletas()
    for jogador in tqdm(lista_mercado_atletas):
        lista_pontuacao = api_cartola.pontuacao_atleta(jogador.id)
        for pontuacao in lista_pontuacao:
            dict_jogador = {
                'apelido': jogador.apelido,
                'clube_id': jogador.clube.id,
                'clube_nome': jogador.clube.nome,
                'jogador_id': jogador.id,
                'posicao_id': jogador.posicao.id,
                'posicao_nome':jogador.posicao.nome,
                'status_id': jogador.status.id,
                'status_nome':jogador.status.nome,
                'scout': jogador.scout,
                'rodada': pontuacao.rodada_id,
                'pontos': pontuacao.pontos,
                'preco': pontuacao.preco,
                'variacao': pontuacao.variacao,
                'media': pontuacao.media
            }
            lista_mercado_atletas_formatado.append(dict_jogador)
    df_jogadores = pd.DataFrame(lista_mercado_atletas_formatado)
    df_jogadores.to_csv(ARQUIVO_JOGADORES)
    print('NOVO ARQUIVO DE PARTIDAS CRIADO')
        
    return df_jogadores

In [22]:
try:
    df_partidas = pd.read_csv(ARQUIVO_PARTIDAS)
    
except:
    df_partidas = cria_tabela_de_partidas(api_cartola)

In [6]:
rodada_atual = descobre_rodada_atual(df_partidas)
rodada_atual

2020-11-26 00:37:29,504 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


24

In [10]:
try:
    df_jogadores = pd.read_csv(ARQUIVO_JOGADORES)
except:
    df_jogadores = cria_tabela_jogadores(api_cartola)


NOVO ARQUIVO DE PARTIDAS CRIADO


# Formata features

In [42]:
df_mergeado = df_jogadores.merge(df_partidas, on=['clube_id','rodada', 'clube_nome'], how='left')

# Features calculadas 
df_mergeado = df_mergeado.sort_values(by=['jogador_id', 'rodada'])
df_mergeado['media_anterior'] = df_mergeado.groupby('jogador_id').media.shift(1).fillna(0)
df_mergeado['pontos_anterior'] = df_mergeado.groupby('jogador_id').pontos.shift(1).fillna(0)
df_mergeado['total_pontos_clube'] = df_mergeado.groupby(by=['clube_nome', 'rodada']).pontos.transform('sum')
df_mergeado['total_pontos_clube_anterior'] = df_mergeado.groupby(by=['clube_nome', 'rodada']).pontos_anterior.transform('sum')

df_dados_clube_adversario = df_mergeado.drop_duplicates(subset=['clube_id', 'rodada'])[['total_pontos_clube_anterior','total_pontos_clube','clube_id', 'rodada']]
df_mergeado = df_mergeado.merge(
    df_dados_clube_adversario,
    left_on = ['clube_adversario_id', 'rodada'],
    right_on = ['clube_id', 'rodada'],
    how='left',
    suffixes=('','_adversario')).drop(columns=['clube_id_adversario'])

df_mergeado = df_mergeado.sort_values(by=['jogador_id', 'rodada'])

lista_novo_df = []
for jogador_id, subdf in df_mergeado.groupby('jogador_id'):
    
    lista_pontos_casa_rodadas_anteriores = []
    lista_pontos_visitante_rodadas_anteriores = []
    
    for linha in subdf.to_dict('records'):
        
        if len(lista_pontos_casa_rodadas_anteriores) == 0:
            linha['media_casa'] = 0
        else:
            linha['media_casa'] = sum(lista_pontos_casa_rodadas_anteriores)/len(lista_pontos_casa_rodadas_anteriores)
        
        if len(lista_pontos_visitante_rodadas_anteriores) == 0:
            linha['media_fora'] = 0
        else:
            linha['media_fora'] = sum(lista_pontos_visitante_rodadas_anteriores)/len(lista_pontos_visitante_rodadas_anteriores)
        
        if (linha['local_partida'] == 'casa') and (linha['pontos']!=0):
            lista_pontos_casa_rodadas_anteriores.append(linha['pontos'])
        if (linha['local_partida'] != 'casa') and (linha['pontos']!=0):
            lista_pontos_visitante_rodadas_anteriores.append(linha['pontos'])

        lista_novo_df.append(linha)  
        
df_mergeado = pd.DataFrame(lista_novo_df)

df_mergeado['media_real'] = df_mergeado.apply(lambda x: x['media_casa'] if x['local_partida']=='casa' else x['media_fora'], axis=1)


df_mergeado.to_csv(ARQUIVO_PROCESSADO, index=False)

# Algoritimo genético

In [43]:
!pip install deap

     |████████████████████████████████| 163kB 5.4MB/s 


In [44]:
from deap import base, creator, tools
import random
import sys
import time


In [62]:
# Filtramos jogadores dessa rodada
df_mergeado_rodada = df_mergeado[df_mergeado.rodada == df_mergeado.rodada.max()]
df_mergeado_rodada = df_mergeado_rodada.fillna(0)

# organizamos os possiveis valores de jogador por posicao
dict_posicao_jogadores = {}
for linha in df_mergeado_rodada[['posicao_id', 'jogador_id']].to_dict('records'):
  dict_posicao_jogadores[linha['posicao_id']] = dict_posicao_jogadores.get(linha['posicao_id'], []) + [linha['jogador_id']]

# Dicionario com atributos
dict_jogadores_atributos = {}
for linha in df_mergeado_rodada.to_dict('records'):
  dict_jogadores_atributos[linha['jogador_id']] =linha

In [103]:
def cria_individuo():
  # Formacao 4 x 3 x 3
  # posicao, numero_jogadores
  formacao = [(1,1),(2,2),(3,2),(4,3),(5,3),(6,1)]

  individuo = []
  for (posicao, numero_jogadores) in formacao:
    individuo += [
          (id, posicao) for id in 
          random.sample(dict_posicao_jogadores[posicao],numero_jogadores)
    ]
        

  # Escolhe capitao
  individuo += [random.choice(individuo)]
  # Um individuo eh formado de lista de tuplas. Cada tupla tem um id de jogador
  # e o id de sua posicao
  return individuo

def evaluate(individual):
    jogadores = individual[0]
    soma_das_medias = sum([dict_jogadores_atributos[i[0]]['media_real'] for i in  jogadores])
    soma_precos = sum([dict_jogadores_atributos[i[0]]['preco'] for i in  jogadores])
    time_compravel = soma_precos <= CARTOLETAS

    diferenca_total_pontos_clube = sum([
                                        (dict_jogadores_atributos[i[0]]['total_pontos_clube'] - dict_jogadores_atributos[i[0]]['total_pontos_clube_adversario']) 
                                        for i in jogadores
                                        ])/len(jogadores) * PESO_DIFERENCAS_TIME
    diferenca_total_pontos_clube_anterior = sum([
                                        (dict_jogadores_atributos[i[0]]['total_pontos_clube_anterior'] - dict_jogadores_atributos[i[0]]['total_pontos_clube_anterior_adversario']) 
                                        for i in jogadores
                                        ])/len(jogadores) * PESO_DIFERENCAS_TIME_ANTERIOR

    funcao_custo = (
        int(soma_das_medias) + 
        float(diferenca_total_pontos_clube) + 
        float(diferenca_total_pontos_clube_anterior)
    ) * int(time_compravel)

    return funcao_custo,

def mutacao(individual):
    # Modificamos o jogador de uma posicao respeitando sua posicao
    id_a_mudar, posicao_alvo = random.choice(individual[0])
    jogadores_possiveis = [
                           i for i in 
                           dict_posicao_jogadores[posicao_alvo]
                           if i not in [i[0] for i in individual[0]]
    ]
    novo_jogador = random.choice(jogadores_possiveis)
    for index, (jogador, posicao) in enumerate(individual[0]):
        if jogador == id_a_mudar:
            jogador = novo_jogador
        individual[0][index] = (jogador,posicao)

    return individual,


def cruzamento(ind1,ind2):

    # Assumimos que ambos possuem a mesma formacao 
    posicoes_possiveis = set([i[1] for i in ind1[0]])

    # capitao
    posicao_capitao_1 = ind1[0][-1][1]
    posicao_capitao_2 = ind2[0][-1][1]

    # possicoes possiveis pra troca
    posicoes_possiveis = posicoes_possiveis - {posicao_capitao_1, posicao_capitao_2}
    posicao_a_trocar = random.choice(list(posicoes_possiveis))

    for index, (i1,i2) in enumerate(zip(ind1[0],ind2[0])):
        if i1[1] == posicao_a_trocar:
            ind1[0][index], ind2[0][index] = ind2[0][index], ind1[0][index]

    return ind1,ind2


In [106]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

GENES = 1
TAM_POP = 100
CXPB = 0.6
MUTPB = 0.8
MAX_GENERATIONS = 10000
CARTOLETAS = 100 # DINHEIRO DISPONIVEL
PESO_DIFERENCAS_TIME = 0.5
PESO_DIFERENCAS_TIME_ANTERIOR = 0.2


toolbox = base.Toolbox()
toolbox.register("attribute", cria_individuo)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attribute, n=GENES)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", cruzamento)
toolbox.register("mutate", mutacao)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("evaluate", evaluate)

/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [107]:
#random.seed(64)

# Atrubuindo tamanho da populaÃ§Ã£o
pop = toolbox.population(n=TAM_POP)

print("Iniciando a Evolucao...")

# Faz um Evaluation com Toda a populaÃ§Ã£o
fitnesses = list(map(toolbox.evaluate, pop))

for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

print("  Individuos avaliados => %i" % len(pop))
print("")

# Inicializa as geraÃ§Ãµes e variaveis
g = 0
max_anterior = 0
contagem = 0

try:

    # Inicia a evoluÃ§Ã£o
    while g < MAX_GENERATIONS:

        # Marca o tempo inicial
        ini = time.time()

        # Cria uma nova GeraÃ§Ã£o
        g = g + 1
        print("-- Geracao %i --" % g)

        # Aplica a seleÃ§Ã£o na prÃ³xima geraÃ§Ã£o de individuos
        descendentes = toolbox.select(pop, len(pop))

        # Clona os individuos selecionados
        descendentes = list(map(toolbox.clone, descendentes))
        
        # Aplica o cruzamento nos descendentes
        for filho1, filho2 in zip(descendentes[::2], descendentes[1::2]):

            # Se uma variÃ¡vel aleatÃ³ria for menor que a taxa de cruzamento
            if random.random() < CXPB:
                # Cruza dois filhos
                toolbox.mate(filho1, filho2)
                # apaga ambos valores de fitness dos filhos
                del filho1.fitness.values
                del filho2.fitness.values

        # Aplica a mutaÃ§Ã£o aos descendentes
        
        for mutante in descendentes:

            # Se uma variÃ¡vel aleatÃ³ria for menor que a taxa de mutaÃ§Ã£o
            if random.random() < MUTPB:
                # Muta o filho
                toolbox.mutate(mutante)
                #Apaga o valor do filho mutante
                del mutante.fitness.values
                
        # Evaluate os individuos com fitness invÃ¡lidos
        invalid_ind = [ind for ind in descendentes if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        #A populaÃ§Ã£o serÃ¡ inteiramente substituida pelos descendentes
        pop[:] = descendentes
        # Salvamos todos os fitness para as estatÃ­sticas
        fits = [ind.fitness.values[0] for ind in pop]

        try:

            length = len(pop)
            mean = sum(fits) / length
            sum2 = sum(x * x for x in fits)
            std = abs(sum2 / length - mean ** 2) ** 0.5

        except ZeroDivisionError:

            print("  Minimo       : 0.0")
            print("  Maximo       : 0.0")
            print("  Media        : 0.0")
            print("  Desvio Padrao: 0.0")

        else:

            print("  Minimo       : %s" % min(fits))
            print("  Maximo       : %s" % max(fits))
            print("  Media        : %s" % mean)
            print("  Desvio Padrao: %s" % std)

            maximo = max(fits)

        if maximo == max_anterior:

            contagem += 1

        else:

            contagem = 0

        if (contagem == 200):
            break

        max_anterior = maximo
        fim = time.time()

        print("Tempo de Execucao - {} Segundos".format(str(fim - ini)))
        print("Numero de Valores Maximos Iguais - {}".format(str(contagem)))
        print("")

except KeyboardInterrupt:

    print("")
    print("-" * 60)
    print("-- Parada Forcada da Evolucao --")
    print("-" * 60)
    print("")
    best_ind = tools.selBest(pop, 1)[0]
    print("O Melhor individuo eh: %s" % (best_ind))
    print("A Melhor populacao eh: %s" % (best_ind.fitness.values))

    # quit
    sys.exit()

print("-- Fim da Evolucao --")

best_ind = tools.selBest(pop, 1)[0]
print("O Melhor individuo eh: %s" % (best_ind))
print("A Melhor resultado da funcao objetivo eh: %s" % (best_ind.fitness.values[0]))


Iniciando a Evolucao...
  Individuos avaliados => 100

-- Geracao 1 --
  Minimo       : 15.020846153846152
  Maximo       : 52.262846153846155
  Media        : 30.702430769230787
  Desvio Padrao: 8.545435502351573
Tempo de Execucao - 0.025682449340820312 Segundos
Numero de Valores Maximos Iguais - 0

-- Geracao 2 --
  Minimo       : 17.28753846153846
  Maximo       : 65.26492307692308
  Media        : 40.181150769230776
  Desvio Padrao: 8.748818329513469
Tempo de Execucao - 0.022733688354492188 Segundos
Numero de Valores Maximos Iguais - 0

-- Geracao 3 --
  Minimo       : 32.47646153846154
  Maximo       : 69.28099999999999
  Media        : 49.81399307692309
  Desvio Padrao: 7.167241380635555
Tempo de Execucao - 0.021512508392333984 Segundos
Numero de Valores Maximos Iguais - 0

-- Geracao 4 --
  Minimo       : 38.78669230769231
  Maximo       : 76.88600000000001
  Media        : 57.14721076923077
  Desvio Padrao: 7.048077432373692
Tempo de Execucao - 0.019857406616210938 Segundos
Num

In [143]:
[dict_jogadores_atributos[i[0]]['apelido'] for i in best_ind[0]]

['Gatito Fernández',
 'Alyson',
 'Zeca',
 'Pedro Geromel',
 'João Victor',
 'Giovanni Augusto',
 'Thiago Maia',
 'Nonato',
 'Bergson',
 'Pedro',
 'Diego Souza',
 'Rogério Ceni',
 'Rogério Ceni']

[[(78584, 1),
  (63013, 2),
  (50645, 2),
  (110672, 3),
  (105282, 3),
  (106199, 4),
  (78865, 4),
  (38196, 4),
  (62128, 5),
  (68835, 5),
  (101398, 5),
  (36943, 6),
  (68835, 5)]]